In [2]:
from dashboardUtils import *

driver = Driver(HEADLESS=True)

In [3]:
def loginToHumhub():
    driver.get('https://hub.e-nable.org/user/auth/login')
    elem = driver.find_element_by_id('login_username')
    elem.send_keys('jschull@e-nable.org')
    sleep(1)
    elem = driver.find_element_by_id('login_password')
    sleep(1)
    elem.send_keys('JcE98vA5\n')
    
loginToHumhub()

In [4]:
sleep(3)
stats = OrderedDict()
driver.get('https://hub.e-nable.org/directory/members?page=1&per-page=25&')
sleep(5)
stats['user-total'] = driver.find_element_by_id('user-total').get_attribute('value')
stats['user-online'] = driver.find_element_by_id('user-online').get_attribute('value')
stats['date']=date()
stats['time']=time()
stats['page']='hub.e-nable.org'
stats



OrderedDict([('user-total', '608'),
             ('user-online', '15'),
             ('date', '2019-12-03'),
             ('time', '12:29'),
             ('page', 'hub.e-nable.org')])

In [5]:
DataFrame([stats])

,user-total,user-online,date,time,page
0,608,15,2019-12-03,12:29,hub.e-nable.org


In [8]:
from pySheets import DictArray
try:
    hubStats = pd.read_excel('hubstats.xls')
    da=DictArray(hubStats)
    da = [stats] + da #newest data first
    hubStats=DataFrame(da)
except FileNotFoundError:
    hubStats = DataFrame([stats])
    
hubStats.to_excel('hubstats.xls', index=False)

In [54]:
def getSpaces(kind='official', URL='https://hub.e-nable.org/enterprise/spacetype/directory/index?id=2'):
    driver.get(URL)
    sleep(1)
    bodies = driver.find_elements_by_class_name('media-body')
    hrefs =[body.find_element_by_tag_name('a').get_attribute('href')
               for body in bodies if body.text]
    kinds = [kind]*len(hrefs)
    return dict(zip(hrefs,kinds))
#getSpaces()

In [55]:
officalSpaces = getSpaces('official', 'https://hub.e-nable.org/enterprise/spacetype/directory/index?id=2')
chapterSpaces = getSpaces('chapter',  'https://hub.e-nable.org/enterprise/spacetype/directory/index?id=4')
projectSpaces = getSpaces('project',  'https://hub.e-nable.org/enterprise/spacetype/directory/index?id=5')
volunteerSpaces=getSpaces('volunteer','https://hub.e-nable.org/enterprise/spacetype/directory/index?id=1')

In [56]:
#merge them
allSpaces = {}
for D in [officalSpaces, chapterSpaces, projectSpaces, volunteerSpaces]:
    for k,v in D.items():
          allSpaces[k] = v


In [57]:
from collections import OrderedDict
spaceDicts = []
for href, kind in allSpaces.items():
    spaceDict = OrderedDict(href=href, kind=kind)
    driver.get(href)
    sleep(1)
    spaceDict['name'] = driver.find_elements_by_class_name('row')[0].text.split('\n')[0]
    statsRow = driver.find_elements_by_class_name('row')[1].text
    statsRow = statsRow.split('\n')
    spaceDict['posts']    = statsRow[0]
    spaceDict['members']  = statsRow[2]
    spaceDict['followers']= statsRow[4]
    #print(spaceDict)
    spaceDicts.append(spaceDict)

TypeError: 'dict_items' object is not subscriptable

In [ ]:
DataFrame(spaceDicts).to_excel('spacestmp.xls')
DataFrame(spaceDicts)

In [59]:
allSpaces

{'https://hub.e-nable.org/s/electronics/': 'official',
 'https://hub.e-nable.org/s/e-nable-spc/': 'official',
 'https://hub.e-nable.org/s/e-nable-chapter-leaders/': 'official',
 'https://hub.e-nable.org/s/device-customization/': 'official',
 'https://hub.e-nable.org/s/e-nable-devices/': 'official',
 'https://hub.e-nable.org/s/e-nable-digital-badge-awards/': 'official',
 'https://hub.e-nable.org/s/e-nable-educators-exchange/': 'official',
 'https://hub.e-nable.org/s/e-nable-forum/': 'official',
 'https://hub.e-nable.org/s/e-nable-forum-archives/': 'official',
 'https://hub.e-nable.org/s/e-nable-fundraising/': 'official',
 'https://hub.e-nable.org/s/e-nable-volunteering-needs/': 'official',
 'https://hub.e-nable.org/s/e-nable-web-central/': 'official',
 'https://hub.e-nable.org/s/e-nablecon-2019/': 'official',
 'https://hub.e-nable.org/s/duke-enable/': 'chapter',
 'https://hub.e-nable.org/s/e-nable-bucknell-university/': 'chapter',
 'https://hub.e-nable.org/s/intero-unam-lab-mexico-city/